## Consistency

- Consistency is the property of a transaction 
- that guarantees that the changes done will be following all the defined rules.

### Database Transaction

- a single change in database
- a transaction can involve multiple operations
- e.g. money transfer (**transaction**):
1. subtraction from buyer
2. addition to seller
- e.g. item transfer

- money and item transfer could be also one transfer

What is a transaction/operation depends on the nature of the application.

## ACID
- The properties 
1. atomic 
2. consistent
3. isolated
4. durable 
are commonly known as the ACID properties transactions.

### Atomicity:

A database transaction must be done as a whole or not done at all.

### Consistency
- Data is in a consistent state when a transaction starts and when it ends.
e.g.an application that transfers funds from one account to another:
the consistency property ensures that

the *total value of funds in both the accounts* is the same at the start and end of each transaction.


### Isolation

- A database transaction must also be isolated.
- A higher level of isolation has higher chances of one transaction blocking another.
- A lower level of isolation has higher chances of producing concurrency undesired effects.

### Durability
- A database transaction must also be durable.
- The changes should be persistent.
- Once a change is done, it is written on the file system
- only another change will remove that change.

### ACID COMPLIANCE
- postgres
- mysql
- sqlite

All Relational DBMS are, or can be, ACID compliant.

Some NoSQL DBMS can be ACID compliant:

- MongoDB
- CouchDB
- Db2

But ACID is a relational concept and most NoSQL use the BASE properties
- BASE properties focus on availability rather than consistency


### BASE Transaction Properties

Most NoSQL databases use transactions that are basically available, soft state and eventually consistent.

- Basically Available
1. The data is spread in different systems to increase availability. 
2. Having data easily available is more important than having a current version of the data.
- Soft State
1. The database does not ensure consistency
2. it is the application developer who must ensure it 
- Eventually Consistent
1. The database will eventually be consistent. 
2. when an update is made in a distributed database, that update will be reflected in all database nodes 
3. that results in the same response every time the data is queried

#### Eventually Consistent
![](event_consistent.png)


ACID versus BASE

- NoSQL databases provide so-called BASE guarantees
- sacrifice a degree of consistency in order to increase availability
- Rather than providing strong consistency, they provide eventual consistency
- a datastore that provides BASE guarantees can occasionally fail to return the result of the latest WRITE

In a typical example, an eventual consistency NoSQL database might permit a hotel room to be reserved on the same night by two different customers, one in Hong Kong and the other in New York City.

For many business applications, this is not a problem.

The resilience, availability, and low-latency afforded by NoSQL are well worth the tradeoff. 

The hotel can provide perks to a disgruntled customer, but, by contrast, a regional outage could create significant churn.




## Database Concurrency
- is the ability to execute two or more transactions at the same time.




### READ vs. write
- Reading transactions can be executed simultaneously.
- Writing transactions very often cannot be executed simultaneously.

The nature of the transactions determines how the concurrent transactions are managed.

### Concurrency Issues
The isolation property prevents the occurrence of some undesired behavior due to concurrent writing transactions.

1. Dirty reads
- A transaction reads data written by concurrent uncommitted transactions.
- we are not sure about the consistency of the data that is read because we don’t know the result of the open transaction
2. Non-repeatable reads
- A transaction re-reads data it has previously read and finds that data has been modified by another transaction
- occurs, when during the course of a transaction, a row is retrieved twice and the values within the row differ between reads.
- user b commits between two queries of user a
3. Phantom reads
- the result changes if executed again, due to another recently-committed transaction.
- All the rows in the query have the same value before and after, but different rows are being selected (because B has deleted or inserted some)
- when reading from committed INSERTS and/or DELETES from another transaction.


### Concurrent Writing Transactions

- Concurrent writing transactions are executed sequentially to prevent isolation issues.
- While the system executes the first transaction the rest stays “on hold”, waiting for the first to finish.

- So how do Write Transaction 2 & 3 know they need to wait?
1. A writing transaction request arrives. It checks if it has access to the data. It does, so it executes.
2. When the transaction is executed, a lock is added to the database.
3. A second transaction request arrives. It checks if it has access to the data. It finds the lock, so it waits.
4. When the first transaction finishes, the lock is released and the next transaction checks again.
5. A COMMIT; and a ROLLBACK; will release the lock.


### Some writing transactions can be done simultaneously.

- Each new transaction has a lock and a key. 
- Checks if the key fits in the current lock.
- If the key fits, adds its own lock and executes.
- A third transaction will try to open every lock it finds on its way to the data. 
- If it succeeds, adds a new lock and executes.

### Lock Modes in PostgreSQL
1. ACCESS SHARE
- It accesses the contents and is compatible with all lock modes, except ACCESS EXCLUSIVE.
2. ACCESS EXCLUSIVE
- guarantees that the current transaction is the only one with any kind of access. 
- (Used by DROP TABLE and other commands.)
3. EXCLUSIVE
- allows only reading access to the table
- (used by REFRESH MATERIALIZED VIEW CONCURRENTLY.)
4. ROW EXCLUSIVE
-  is used by UPDATE, DELETE and INSERT operations. 
- It conflicts with SHARE, SHARE ROW EXCLUSIVE, EXCLUSIVE, and ACCESS EXCLUSIVE lock modes.
5. SHARE
- is triggered by CREATE INDEX. 
- Conflicts with ROW EXCLUSIVE and all lock modes used by DDL commands

### Locking in Postgres
- Locks and keys are managed with Lock Modes. 
- The most common ones are used on tables.

1. select
- only blocks some ALTER TABLE and all DROP TABLE and VACUUM commands.
2. update, insert, delete
- block the same as the SELECT, plus CREATE INDEX.
3. alter table
- may use different locks depending on what it does. 
- Some of them use the ACCESS EXCLUSIVE lock mode.
